<a href="https://colab.research.google.com/github/MeenalShah13/SentimentAnalysisCOVID/blob/main/Sentiment_classifier_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
###!wget https://github.com/MeenalShah13/SentimentAnalysisCOVID/sample_data.zip

--2021-04-05 16:53:25--  https://github.com/MeenalShah13/SentimentAnalysisCOVID/sample_data.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-04-05 16:53:25 ERROR 404: Not Found.



In [4]:
!unzip sample_data.zip

Archive:  sample_data.zip
  inflating: sample_data.csv         


In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('sample_data.csv')
df.head()

,TweetID,Username,TimeStamp,Follower_num,Friend_num,Retweets_num,Favourites_num,Tweet,Sentiment,Mentions,Hashtags,URL
0,1338226694444859394,089be85f6a89c59e88023f993960401a,Sun Dec 13 20:58:01 +0000 2020,188,199,0,0,null;,1 -4,MikeEmanuelFox,null;,null;
1,1300246448995409924,7b32b6b1251ff68e93fbca72ac103d28,Mon Aug 31 01:38:05 +0000 2020,1524,323,5,10,null;,1 -3,johncardillo,null;,null;
2,1278708694390173696,43250c612d875e4b4723f8bc9267f4e4,Thu Jul 02 15:14:44 +0000 2020,4018,4357,0,0,claw back:Clawback:-2.8650884132444534;,1 -1,null;,null;,https://www.wsj.com/articles/businesses-hit-ha...
3,1321386099785293824,47274922378102842a9d0d3864a9f732,Wed Oct 28 09:39:31 +0000 2020,744,732,9,17,where i live:Where_I_Live:-2.284433538491413;,1 -3,confidencenac,null;,null;
4,1296079483523522568,a66b8dc9de405ca8dd7616b74de4c3f8,Wed Aug 19 13:40:03 +0000 2020,15694,30,2,5,tu quoque:Tu_quoque:-1.1387633458589146;,2 -1,null;,null;,null;


In [4]:
df.shape

(150000, 12)

In [5]:
df.isnull().sum() 

TweetID             0
Username            0
TimeStamp           0
Follower_num        0
Friend_num          0
Retweets_num        0
Favourites_num      0
Tweet               0
Sentiment           0
Mentions          270
Hashtags           58
URL                 0
dtype: int64

In [6]:
df = df.dropna()

In [7]:
df.duplicated().sum()

0

In [8]:
df1 = df.drop(['TweetID','Username','Follower_num','Friend_num','Retweets_num','Favourites_num','Mentions','URL','Hashtags'],axis=1)
df1.head()

,TimeStamp,Tweet,Sentiment
0,Sun Dec 13 20:58:01 +0000 2020,null;,1 -4
1,Mon Aug 31 01:38:05 +0000 2020,null;,1 -3
2,Thu Jul 02 15:14:44 +0000 2020,claw back:Clawback:-2.8650884132444534;,1 -1
3,Wed Oct 28 09:39:31 +0000 2020,where i live:Where_I_Live:-2.284433538491413;,1 -3
4,Wed Aug 19 13:40:03 +0000 2020,tu quoque:Tu_quoque:-1.1387633458589146;,2 -1


In [9]:
df1.shape

(149672, 3)

In [10]:
blanks=[]
for i,time,tweet,senti in df1.itertuples():  # iterate over the DataFrame
    if type(tweet)==str:            # avoid NaN values
        if tweet=='null;':         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

df1.drop(blanks, inplace=True)
df1.shape

(103469, 3)

In [11]:
df1.head()

,TimeStamp,Tweet,Sentiment
2,Thu Jul 02 15:14:44 +0000 2020,claw back:Clawback:-2.8650884132444534;,1 -1
3,Wed Oct 28 09:39:31 +0000 2020,where i live:Where_I_Live:-2.284433538491413;,1 -3
4,Wed Aug 19 13:40:03 +0000 2020,tu quoque:Tu_quoque:-1.1387633458589146;,2 -1
5,Fri Sep 04 09:44:29 +0000 2020,sarkodie:Sarkodie_%28rapper%29:-1.195632594385...,2 -1
6,Thu Oct 29 20:12:39 +0000 2020,kidding:Kidding:-2.9308236049363066;,2 -3


In [12]:
df1['score']  = df1['Sentiment'].apply(lambda s:  s.split())
df1['score'] = df1['score'].apply(lambda x :int(x[0])+int(x[1]))


In [13]:
df1.head()

,TimeStamp,Tweet,Sentiment,score
2,Thu Jul 02 15:14:44 +0000 2020,claw back:Clawback:-2.8650884132444534;,1 -1,0
3,Wed Oct 28 09:39:31 +0000 2020,where i live:Where_I_Live:-2.284433538491413;,1 -3,-2
4,Wed Aug 19 13:40:03 +0000 2020,tu quoque:Tu_quoque:-1.1387633458589146;,2 -1,1
5,Fri Sep 04 09:44:29 +0000 2020,sarkodie:Sarkodie_%28rapper%29:-1.195632594385...,2 -1,1
6,Thu Oct 29 20:12:39 +0000 2020,kidding:Kidding:-2.9308236049363066;,2 -3,-1


In [14]:
df1['label'] = df1['score'].apply(lambda c: 'pos' if c >0 else ('neu' if c==0 else 'neg'))

In [15]:
df1.head()

,TimeStamp,Tweet,Sentiment,score,label
2,Thu Jul 02 15:14:44 +0000 2020,claw back:Clawback:-2.8650884132444534;,1 -1,0,neu
3,Wed Oct 28 09:39:31 +0000 2020,where i live:Where_I_Live:-2.284433538491413;,1 -3,-2,neg
4,Wed Aug 19 13:40:03 +0000 2020,tu quoque:Tu_quoque:-1.1387633458589146;,2 -1,1,pos
5,Fri Sep 04 09:44:29 +0000 2020,sarkodie:Sarkodie_%28rapper%29:-1.195632594385...,2 -1,1,pos
6,Thu Oct 29 20:12:39 +0000 2020,kidding:Kidding:-2.9308236049363066;,2 -3,-1,neg


In [16]:
df1['label'].value_counts()

neu    43266
pos    31864
neg    28339
Name: label, dtype: int64

In [17]:
import re
contractions_dict = {"can`t":"can not",
                     "won`t":"will not",
                     "don`t":"do not",
                     "aren`t":"are not",
                     "i`d":"i would",
                     "couldn`t": "could not",
                     "shouldn`t": "should not",
                     "wouldn`t": "would not",
                     "isn`t": "is not",
                     "it`s": "it is",
                     "didn`t": "did not",
                     "weren`t": "were not",
                     "mustn`t": "must not",
                    }

def prepare_data(df1:pd.DataFrame) -> pd.DataFrame:
    
    df1["text"] = df1["text"] \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))
        
    df1["label"] = df1["label"].map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )
    return df1.text.values, df1.label.values
def replace_words(string:str, dictionary:dict):
    for k, v in dictionary.items():
        string = string.replace(k, v)
    return string


In [18]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
sample = df1.sample(n=15000)
X = sample.Tweet
Y =sample.label

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



X_train = X_train \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))

X_test = X_test \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))

y_train =  y_train.map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )
y_test = y_test.map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )


(12000,) (12000,)
(3000,) (3000,)


In [19]:
train_tweets, train_labels = X_train,y_train
test_tweets, test_labels = X_test,y_test

In [1]:
pip install keras

In [20]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tweets)
train_tokenized = tokenizer.texts_to_matrix(
                             train_tweets,
                             mode='tfidf'
                             )
test_tokenized = tokenizer.texts_to_matrix(
                             test_tweets,
                             mode='tfidf'
                             )

In [21]:
from  sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(
                                n_estimators=500, 
                                min_samples_leaf=2,
                                oob_score=True,
                                n_jobs=-1,
                                )
forest.fit(train_tokenized,train_labels)
print(f"Train score: {forest.score(train_tokenized,train_labels)}")
print(f"OOB score: {forest.oob_score_}")

Train score: 0.65825
OOB score: 0.44658333333333333


In [22]:
print("Test score:", forest.score(test_tokenized,test_labels))

Test score: 0.4603333333333333


##Vader benchmark